In [1]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

✅ Logging configured


In [2]:
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner, InMemoryRunner
from google.adk.plugins.logging_plugin import (LoggingPlugin)
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.tools.tool_context import ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

print("Sessions and context engineering (3a) and MAS (1b) and Tools (2a) ADK imports")

Sessions and context engineering (3a) and MAS (1b) and Tools (2a) ADK imports


In [4]:
APP_NAME = "Wellness Suite"
USER_ID = "default"
SESSION = "default"
# USER_PROFILE = {
#     "name": USER_ID,
#     "age": None,
#     "weight": None,
#     "height": None,
#     "goals": [],
#     "injuries": [],
#     "dietary_restrictions": []
# }

MODEL_NAME = "gemini-2.5-flash-lite"

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=7, # Delay multiplier
    initial_delay=1, # initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [6]:
# Create a SQLite database with DatabaseSessionService
db_url = "sqlite:///agentic_wellness_data.db"

# Set up Session Management
session_service = DatabaseSessionService(db_url=db_url)

In [7]:
# Helper functions for running a session
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")

In [8]:
# Session state management tools
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")

# 'user:' prefix indicates the following is user-specific data
def save_userinfo(
    tool_context: ToolContext, user_name: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name in session state.

    Args:
        user_name: The username to store in session state
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"] = user_name

    return {"status": "success"}

# tools can read from session state. example:
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user name from session state.
    """

    user_name = tool_context.state.get("user:name", "Username not found")

    return {"status": "success", "user_name": user_name}

In [9]:
# Specialist Agents

# Fitness Trainer Agent: focuses on fitness
fitness_trainer_agent = LlmAgent(
    name="FitnessTrainer",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert fitness trainer with deep knowledge of human physiology.
    Please provide a detailed fitness training plan for the user that is customized for the following:
    1. Their current level of fitness
    2. To help them achieve their fitness goals
    3. Takes into consideration their injury history
    4. Sets realistic time frames for developing lifting technique and expertise
    """,
    tools=[],
    output_key="fitness_plan"
)

# Nutritionist Agent: focuses on nutrition
nutritionist_agent = LlmAgent(
    name="Nutritionist",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert nutritionist with deep knowledge of diet and human metabolism.
    Please provide a detailed nutrition plan for the user that is customized for the following:
    1. Their dietary restrictions
    2. To help them achieve their weight and fitness goals
    3. Suggest meal plans that align with their preferences
    4. Ensure that the nutrition plan meets the macronutrient and micronutrient needs to achieve their goals
    """,
    tools=[],
    output_key="nutrition_plan"
)

In [10]:
# Aggregator Agent

aggregator_agent = LlmAgent(
    name="WellnessAggregator",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""Combine the fitness plan and nutrition plan into a single wellness plan tailored for the user.
    Focus on the synergy between the fitness and nutrition aspects to maximize the user's health and wellness outcomes.

    **Fitness Plan:**
    {fitness_plan}

    **Nutrition Plan:**
    {nutrition_plan}

    Your final wellness plan should seamlessly integrate both components and provide clear, actionable recommendations.
    """,
    tools=[retrieve_userinfo],
    output_key="wellness_plan"
)

In [11]:
# Parallel Agent to run both specialists simultaneously
parallel_wellness_team = ParallelAgent(
    name="ParallelWellnessTeam",
    sub_agents=[fitness_trainer_agent, nutritionist_agent],
)

In [12]:
# Sequential Agent to run the parallel team first, then run the aggregator
sequential_wellness = SequentialAgent(
    name="WellnessSystem",
    sub_agents=[parallel_wellness_team, aggregator_agent],
)

In [13]:
# Intake Agent

intake_agent = LlmAgent(
    name="WellnessIntakeAgent",
    model=Gemini(
        model_name=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert wellness assistant agent with concise yet kind verbiage.
    Ensure that the user provides necessary personal information for fitness and nutrition advice by asking questions about their health.
    Craft your questions to collect the most essential information (e.g. weight, height, dietary preferences, injury history, general fitness level).
    """,
    tools=[retrieve_userinfo],
    output_key="intake_questions",
)

In [14]:
# Centralized root Wellness Coordinator Agent to provide holistic wellness plan

root_agent = LlmAgent(
    name="WellnessCoordinatorAgent",
    model=Gemini(
        model_name=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are the Wellness Coordinator Agent.
    Your goal is to assist users in improving their overall wellness by orchestrating the following workflow.
    1. First, ascertain whether or not you have the user's necessary personal information to provide customized fitness and nutrition advice.
    2. If not, call the `intake_agent` tool to craft questions to obtain the necessary personal information.
    3. Once you have sufficient information, call the `wellness_pipeline` tool to create a wellness plan for the user.
    4. Present the results of the wellness plan to the user.
    """,
    tools=[
        save_userinfo,
        retrieve_userinfo,
        AgentTool(intake_agent),
        AgentTool(sequential_wellness)
    ],
)

In [15]:
# App with Events Compaction enabled
wellness_app_compacting = App(
    name="wellness_app_coompacting",
    root_agent=root_agent,
    plugins=[LoggingPlugin()], 
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=2, # Trigger compaction every 2 invocations
        overlap_size=1, # Keep 1 previous turn for context
    ),
)

/var/folders/p1/bgzqkn9s51lc2t_bx4c0djmr0000gn/T/ipykernel_1564/2613804162.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


In [16]:
wellness_runner_compacting = Runner(app=wellness_app_compacting, session_service=session_service)

In [17]:
await run_session(
    wellness_runner_compacting,
    [
        """Hello! My name is Hector.
        """
    ],
    "test-db-session-02",
)


 ### Session: test-db-session-02

User > Hello! My name is Hector.
        
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-6e05e796-a206-44e2-87b0-61f5e8027454
[logging_plugin]    Session ID: test-db-session-02
[logging_plugin]    User ID: default
[logging_plugin]    App Name: wellness_app_coompacting
[logging_plugin]    Root Agent: WellnessCoordinatorAgent
[logging_plugin]    User Content: text: 'Hello! My name is Hector.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-6e05e796-a206-44e2-87b0-61f5e8027454
[logging_plugin]    Starting Agent: WellnessCoordinatorAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: WellnessCoordinatorAgent
[logging_plugin]    Invocation ID: e-6e05e796-a206-44e2-87b0-61f5e8027454
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash
[logging_plugin]    Agent: WellnessCoordinatorAgent
[logging_plugin]    System Instruction: 'You are the Wellness Coordinator 

In [18]:
await run_session(
    wellness_runner_compacting,
    [
        """I am 30 years old, 5 foot 6 inches tall, and 170 pounds.
        I am currently sedentary but have prior powerlifting experience (about 7 years ago).
        I have no particular dietary preferences or restrictions.
        I tore my left rotator cuff and had it surgically repaired.
        """
    ],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > I am 30 years old, 5 foot 6 inches tall, and 170 pounds.
        I am currently sedentary but have prior powerlifting experience (about 7 years ago).
        I have no particular dietary preferences or restrictions.
        I tore my left rotator cuff and had it surgically repaired.
        
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-61b8f1fb-f142-48ea-a2bd-dbbbb1dff0bb
[logging_plugin]    Session ID: test-db-session-01
[logging_plugin]    User ID: default
[logging_plugin]    App Name: wellness_app_coompacting
[logging_plugin]    Root Agent: WellnessCoordinatorAgent
[logging_plugin]    User Content: text: 'I am 30 years old, 5 foot 6 inches tall, and 170 pounds.
        I am currently sedentary but have prior powerlifting experience (about 7 years ago).
        I have no particular dietary preferences o...'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-61b8f1fb-f142-48ea-a2bd-dbbb

In [19]:
await run_session(
    wellness_runner_compacting,
    [
        """I wish to improve my overall fitness by losing weight while maintaining or building muscle mass and strength.
        """
    ],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > I wish to improve my overall fitness by losing weight while maintaining or building muscle mass and strength.
        
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-f16df921-80c5-4aec-84d1-2bf43ebcbbae
[logging_plugin]    Session ID: test-db-session-01
[logging_plugin]    User ID: default
[logging_plugin]    App Name: wellness_app_coompacting
[logging_plugin]    Root Agent: WellnessCoordinatorAgent
[logging_plugin]    User Content: text: 'I wish to improve my overall fitness by losing weight while maintaining or building muscle mass and strength.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-f16df921-80c5-4aec-84d1-2bf43ebcbbae
[logging_plugin]    Starting Agent: WellnessCoordinatorAgent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: WellnessCoordinatorAgent
[logging_plugin]    Invocation ID: e-f16df921-80c5-4aec-84d1-2bf43ebcbbae
[logging_plugin] 🧠 LLM REQUEST
[l

In [20]:
# Final session state
final_session = await session_service.get_session(
    app_name=wellness_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="test-db-session-01",
)

found_summary = False
for event in final_session.events:
    if event.actions and event.actions.compaction:
        print("Found Compaction Event")
        print(f"\n Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print("\n No compaction event found.")

Found Compaction Event

 Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='998fe9aa-b2b7-4277-9af9-84766e8b2190' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1764281223.844339, 'end_timestamp': 1764281263.562783, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_data': None, 'text': 'The user, a 30-year-old male (5\'6", 170 lbs) w

In [21]:
if os.path.exists("agentic_wellness_data.db"):
    os.remove("agentic_wellness_data.db")
print("Cleaned up old database files")

Cleaned up old database files
